# India Kanoon Companion : AI Legal Assistant

### Installing necessary Libraries

In [1]:
!pip install langchain deeplake openai tiktoken

### Setting up OPENAI_API_KEY & ACTIVELOOP_TOKEN

In [2]:
import os
os.environ['OPENAI_API_KEY']='sk-proj-dajkjkkjldfskldfslk329348989093290CQ1mjekfhZFz'

os.environ['ACTIVELOOP_TOKEN']='eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.fjkdkjdfkjj80948095erwjkfdkml4ti90GRMbzIzcVVWTjJlIn0.'

### Importing necessary Libraries

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain import PromptTemplate

### Splitting by Words

In [4]:
import os

# Initialize an empty list to store words
words = []

# Load the legal document from the .txt file
with open('Combined_PDF2Text_Book.txt', 'r', encoding='utf-8') as file:
    # Read the entire content of the file
    legal_text = file.read()

    # Split the text into words
    words = legal_text.split()

# Now 'words' contains a list of all words in the document
print(f"Total words in the document: {len(words)}")


Total words in the document: 40177


In [5]:
words[:10]

['Why',
 'Know',
 'the',
 'Law',
 'Understanding',
 'the',
 'basics',
 'of',
 'Indian',
 'law']

### Splitting by Sentences

In [6]:
import nltk
from nltk.tokenize import sent_tokenize

# Download the necessary NLTK data files (only needs to be done once)
nltk.download('punkt')

# Load the legal document from the .txt file
with open('Combined_PDF2Text_Book.txt', 'r', encoding='utf-8') as file:
    legal_text = file.read()

# Split the text into sentences
sentences = sent_tokenize(legal_text)

# 'sentences' now contains a list of sentences from the document
print(f"Total sentences in the document: {len(sentences)}")

Total sentences in the document: 2660


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sifta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
sentences[:5]

['\n\nWhy Know the Law  Understanding the basics of Indian law is essential for every common man, as it empowers individuals to know their rights and obligations, enabling them to get along with the society.',
 'This book aims to make Indian law simple and clear for everyone.',
 'We want people to understand how the law works and its importance.',
 'As you progress through this book, you will gain insights into the various aspects of Indian law, empowering you to make informed decisions and become a responsible citizen.',
 "So, let's embark upon this journey to explore and understand the basics of Indian law together."]

### Split by Paragraphs

In [8]:
# Load the legal document from the .txt file
with open('Combined_PDF2Text_Book.txt', 'r', encoding='utf-8') as file:
    legal_text = file.read()

# Split the text into paragraphs
paragraphs = legal_text.split('\n\n')  # Assumes paragraphs are separated by double newlines

# 'paragraphs' now contains a list of paragraphs from the document
print(f"Total paragraphs in the document: {len(paragraphs)}")

# Each paragraph can now be used for further processing in question-answering system


Total paragraphs in the document: 17


In [9]:
paragraphs[:2]

['',
 "Why Know the Law  Understanding the basics of Indian law is essential for every common man, as it empowers individuals to know their rights and obligations, enabling them to get along with the society.  This book aims to make Indian law simple and clear for everyone.  We want people to understand how the law works and its importance.  As you progress through this book, you will gain insights into the various aspects of Indian law, empowering you to make informed decisions and become a responsible citizen.  So, let's embark upon this journey to explore and understand the basics of Indian law together.  A simple Law guide for every Indian  Sree Krishna Seelam  \x0c  Knowing the law helps:  ● Know your rights and duties: By knowing the law, you understand your rights and can protect yourself. You also get to know what duties you should perform as a citizen.  ● Make correct choices: Understanding the law helps you make correct decisions in life, like in relationships, jobs, making p

### Based on the above three splittings, it is better to use sentences for our use case

In [10]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

# Convert each chunk into a Document object
documents = [Document(page_content=chunk) for chunk in sentences]

# Initialize CharacterTextSplitter with desired chunk size and overlap
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=300)

# Split the documents into smaller chunks if necessary
# This step might not split further if the documents are already small enough
split_docs = text_splitter.split_documents(documents)

# Optionally, print out the split documents to verify
for i, doc in enumerate(split_docs):
    print(f"Document {i+1}:")
    print(doc.page_content[:200])  # Print the first 200 characters to check
    print("\n")
    if i==10:
        break


Document 1:
Why Know the Law  Understanding the basics of Indian law is essential for every common man, as it empowers individuals to know their rights and obligations, enabling them to get along with the society


Document 2:
This book aims to make Indian law simple and clear for everyone.


Document 3:
We want people to understand how the law works and its importance.


Document 4:
As you progress through this book, you will gain insights into the various aspects of Indian law, empowering you to make informed decisions and become a responsible citizen.


Document 5:
So, let's embark upon this journey to explore and understand the basics of Indian law together.


Document 6:
A simple Law guide for every Indian  Sree Krishna Seelam  
  Knowing the law helps:  ● Know your rights and duties: By knowing the law, you understand your rights and can protect yourself.


Document 7:
You also get to know what duties you should perform as a citizen.


Document 8:
● Make correct choices: Underst

### Total length of splitted documents/Sentences in the corpus

In [11]:
len(split_docs)

2660

### Initializing OpenAIEmebdding and storing documents embeddings in the DeepLake vectore DB

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# creating Deep Lake dataset
my_activeloop_org_id = "siftain"
my_activeloop_dataset_name = "Indian_Laws_For_Common_People"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# Adding documents to the Deep Lake dataset
# This step processes each document separately to avoid rate limits
batch_size = 128 # Defining a batch size to avoid rate limits
for i in range(0, len(split_docs), batch_size):
    batch = split_docs[i:i+batch_size]
    db.add_documents(batch)

Your Deep Lake dataset has been successfully created!


Creating 128 embeddings in 1 batches of size 128:: 100%|█████████████████████████████████| 1/1 [00:44<00:00, 44.19s/it]


Dataset(path='hub://siftain/Indian_Laws_For_Common_People', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (128, 1)      str     None   
 metadata     json      (128, 1)      str     None   
 embedding  embedding  (128, 1536)  float32   None   
    id        text      (128, 1)      str     None   


Creating 128 embeddings in 1 batches of size 128:: 100%|█████████████████████████████████| 1/1 [00:47<00:00, 47.31s/it]


Dataset(path='hub://siftain/Indian_Laws_For_Common_People', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape      dtype  compression
  -------    -------     -------    -------  ------- 
   text       text      (256, 1)      str     None   
 metadata     json      (256, 1)      str     None   
 embedding  embedding  (256, 1536)  float32   None   
    id        text      (256, 1)      str     None   


...

Dataset(path='hub://siftain/Indian_Laws_For_Common_People', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (2432, 1)      str     None   
 metadata     json      (2432, 1)      str     None   
 embedding  embedding  (2432, 1536)  float32   None   
    id        text      (2432, 1)      str     None   


Creating 128 embeddings in 1 batches of size 128:: 100%|█████████████████████████████████| 1/1 [01:23<00:00, 83.15s/it]


Dataset(path='hub://siftain/Indian_Laws_For_Common_People', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (2560, 1)      str     None   
 metadata     json      (2560, 1)      str     None   
 embedding  embedding  (2560, 1536)  float32   None   
    id        text      (2560, 1)      str     None   


Creating 100 embeddings in 1 batches of size 100:: 100%|█████████████████████████████████| 1/1 [01:18<00:00, 78.38s/it]

Dataset(path='hub://siftain/Indian_Laws_For_Common_People', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
   text       text      (2660, 1)      str     None   
 metadata     json      (2660, 1)      str     None   
 embedding  embedding  (2660, 1536)  float32   None   
    id        text      (2660, 1)      str     None   


### Updating the prompt for a chatbot with memory

In [13]:

template = """
You are a knowledgeable legal assistant. You have access to the following context information:

{chunks_formatted}

Based on the user's current question and the previous conversation, provide a clear and accurate answer.

Previous Conversation: {conversation_history}

User Question: {query}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "conversation_history", "query"],
    template=template,
)


### Initializing an empty list to store conversation history

In [14]:
conversation_history = []

### Chatbot Interaction

In [15]:
def handle_chatbot_interaction(query):
    """
    Handles chatbot interaction, remembering the conversation history.
    
    Parameters:
    - query (str): The current question from the user.
    - temperature (float): The temperature for the LLM model.
    """
    # Retrieve relevant chunks from the database
    docs = db.similarity_search(query)
    retrieved_chunks = [doc.page_content for doc in docs]

    # Format the conversation history
    conversation_formatted = "\n\n".join(conversation_history)
    chunks_formatted = "\n\n".join(retrieved_chunks)

    # Format the prompt
    prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, conversation_history=conversation_formatted, query=query)

    # Generate the answer
    llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
    answer = llm(prompt_formatted)

    # Add the current interaction to the conversation history
    conversation_history.append(f"User: {query}\nAI: {answer}\n")

    # Return the answer
    return answer

# Example interaction loop
while True:
    query = input("Ask your legal question: ")
    
    response = handle_chatbot_interaction(query)
    print("\nAI Response:\n", response)
    
    continue_chat = input("Do you want to ask another question? (yes/no): ").strip().lower()
    if continue_chat != 'yes':
        break


Ask your legal question:  What are fundamental rights?



AI Response:
 Fundamental Rights are essential rights enshrined in Part III of the Constitution of India, which guarantees civil liberty (freedom) and protects individuals from misuse of the power by government. There are six fundamental rights that anchor the Constitution of India, including the right to equality, right to freedom, right against exploitation, right to freedom of religion, cultural and educational rights, and right to constitutional remedies. These rights are considered to be the cornerstone of democracy and are meant to ensure that every citizen of India has equal opportunities and protection under the law.


Do you want to ask another question? (yes/no):  yes
Ask your legal question:  Write me all the fundamental rights?



AI Response:
 
There are six fundamental rights enshrined in Part III of the Constitution of India. These include the right to equality, right to freedom, right against exploitation, right to freedom of religion, cultural and educational rights, and right to constitutional remedies. These rights are considered to be the cornerstone of democracy and are meant to ensure that every citizen of India has equal opportunities and protection under the law.


Do you want to ask another question? (yes/no):  no


In [17]:
# Example interaction loop
while True:
    query = input("Ask your legal question: ")
    
    response = handle_chatbot_interaction(query)
    print("\nAI Response:\n", response)
    
    continue_chat = input("Do you want to ask another question? (yes/no): ").strip().lower()
    if continue_chat != 'yes':
        break

Ask your legal question:  Tell me about the laws regarding Education?



AI Response:
 
One of the laws regarding education is the Right to Establish and Administer Educational Institutions, which is outlined in Article 30 of the Indian Constitution. This article allows religious and linguistic minorities to create and manage their own educational institutions according to their choice, as long as they follow sensible rules. This means that these minorities have the right to establish and run their own schools, colleges, and other educational institutions without interference from the government.


Do you want to ask another question? (yes/no):  no


In [23]:
# Example interaction loop
while True:
    query = input("Ask your legal question: ")
    
    response = handle_chatbot_interaction(query)
    print("\nAI Response:\n", response)
    
    continue_chat = input("Do you want to ask another question? (yes/no): ").strip().lower()
    if continue_chat != 'yes' :
        break

Ask your legal question:  Tell me about Labour and Employment Law.



AI Response:
 
Labour and Employment Law is a branch of law that deals with the rights and responsibilities of employers and employees in the workplace. It covers a wide range of topics such as wages, working conditions, discrimination, and workplace safety. Understanding these laws is important in creating fair and respectful workplaces for everyone at the federal, state, and local levels.


Do you want to ask another question? (yes/no):  yes
Ask your legal question:  What are the different laws for it?



AI Response:
 
The specific laws for a particular subject can vary depending on the state and level of government. It is important to consult local laws and court rules for clarity on the specific laws that apply to your situation. Each level of government has its own power to make laws for certain subjects.


Do you want to ask another question? (yes/no):  yes
Ask your legal question:  What is the law which encounters discrimination in work place?



AI Response:
 The law that addresses discrimination in the workplace is the Protection against discrimination Act, which was enacted in 2013. This law was influenced by the Vishaka v. State of Rajasthan case, which resulted in guidelines for addressing workplace sexual harassment. The Act also specifically bans gender-based wage discrimination.


Do you want to ask another question? (yes/no):  no


In [24]:
# Example interaction loop
while True:
    query = input("Ask your legal question: ")
    
    response = handle_chatbot_interaction(query)
    print("\nAI Response:\n", response)
    
    continue_chat = input("Do you want to ask another question? (yes/no): ").strip().lower()
    if continue_chat != 'yes' :
        break

Ask your legal question:  I want to know about how my college spend the money taken by students, Is there any law for it?



AI Response:
 
Yes, there are laws that regulate how colleges can spend the money taken from students. One example is the Higher Education Act, which requires colleges to use federal student aid funds for educational purposes only. Additionally, there may be state laws or regulations that govern how colleges can use student funds. It is important to research and understand these laws to ensure that your college is using student funds appropriately.


Do you want to ask another question? (yes/no):  no


### If time permits, I Will make different domain specific projects related to Indian Laws.

# Thank You